In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
file = "data/exoplanet.eu_catalog.csv"

In [3]:
data = pd.read_csv(file, skiprows=0, usecols=['radius', 'orbital_period', 'ra', 'dec', 'mag_v', 
                                              'star_distance', 'star_radius', 'semi_major_axis',
                                              'eccentricity', 'tzero_tr'])

In [4]:
# We want to drop rows with null planet and stellar radii
# We also need the V-band magnitudes
data = data.dropna(subset={"radius", "star_radius", "mag_v", "tzero_tr"})
# We want only stars with mag_v ~< 12
data = data[data["mag_v"] < 12]

# Now compute transit depth for each
transit_depth = (0.10049*data["radius"] / data["star_radius"])**2
# Compute transit signature in magnitudes
transit_sig = -2.5 * np.log10(1 - transit_depth)
# Drop all rows where transit signature is < 0.008
data = data[transit_sig > 0.008]

In [5]:
semi_minor_axis = data["semi_major_axis"] * np.sqrt(1 - data["eccentricity"]**2)
# This is the lower bound on circumference
circ = np.sqrt(2) * np.pi * np.sqrt(data["semi_major_axis"]**2 + semi_minor_axis**2)
# Velocity in meters per hour
veloc = circ / data["orbital_period"] * 1.496e11 / 24
# Very rough duration estimate
transit_dur = 2 * data["star_radius"] * 6.957e8 / veloc
# Keep transit duration under 3 hours
data = data[transit_dur < 3]

In [6]:
data = data[data["dec"] > -19]

In [7]:
def from_jd(jd):
    
    A = jd // 1
    L = A+68569
    N = 4*L//146097
    L = L-(146097*N+3)//4
    I = 4000*(L+1)//1461001
    L = L-1461*I//4+31
    J = 80*L//2447
    K = L-2447*J//80
    L = J//11
    J = J+2-12*L
    I = 100*(N-49)+I+L
    
    B = jd % 1
    hour = B * 24
    minute = (hour % 1) * 60
    second = (minute % 1) * 60
    
    I = map(int, I)
    J = map(int, J)
    K = map(int, K)
    hour = map(int, hour)
    minute = map(int, minute)
    second = map(int, second)
    
    return [datetime(i, j, k, h, m, s) for i, j, k, h, m, s
            in zip(I, J, K, hour, minute, second)]

def equinox(year):
    
    return datetime(year, 3, 21, 0, 0, 0)

def delta_ra(dates):
    
    dts = (dateobj - equinox(dateobj.year) for dateobj in dates)
    return np.array([dt.days for dt in dts])

# ra = (data["ra"] + 180) % 360
# data = data[ra > 285]
dra = delta_ra(from_jd(data["tzero_tr"]))
ra = dra * 360 / 365.25

In [8]:
# September 5th at 19:00
after = 2458731.29167
# Compute the next transit
rem = (after - data["tzero_tr"]) % data["orbital_period"]
obsday = after - rem + data["orbital_period"]

for i in obsday.index:
    
    while not (0.3 <= obsday[i] % 1 <= 0.5): obsday[i] += data["orbital_period"][i]
        
print(*obsday)

2458746.4527950003 2458733.49953 2458735.4547449993 2458746.34906 2458733.4934400003 2458739.4034970007 2458734.3312636795 2458790.3089429974 2458780.319621202 2458733.3311515553 2458737.300653599 2458734.34872 2458739.3231243985 2458733.463808 2458739.31579664 2458731.3300093003 2458731.3934548004 2458742.390650001 2458741.39584701 2458743.4631911013 2458754.376821401 2458745.361751601 2458759.4568967987 2458733.313034


In [9]:
data  

,radius,orbital_period,semi_major_axis,eccentricity,tzero_tr,ra,dec,mag_v,star_distance,star_radius
312,1.045,3.722469,0.04390,0.0000,2.456308e+06,47.370833,30.673611,11.890,125.0,0.810
320,1.289,2.775960,0.04130,0.0360,2.455028e+06,9.575000,42.463056,10.800,235.0,1.237
326,0.867,2.875317,0.03610,0.0150,2.456708e+06,111.916667,24.336389,11.340,70.0,0.694
328,1.080,3.212220,0.04140,0.0160,2.456707e+06,155.683333,50.128333,9.730,82.0,1.040
329,1.368,1.212884,0.02320,0.1060,2.454852e+06,306.125000,16.762222,11.940,393.0,1.290
336,0.827,2.899703,0.03866,0.0000,2.454857e+06,206.095833,48.028611,11.860,130.0,0.799
695,1.138,2.218573,0.03142,0.0041,2.453989e+06,300.179167,22.710833,7.670,19.3,0.805
3773,1.062,3.506189,0.04230,0.0000,2.457784e+06,222.208333,22.152500,11.400,101.0,0.722
3825,1.099,3.030072,0.03930,0.0000,2.453187e+06,286.037500,36.632500,11.790,157.0,0.850
3826,1.189,2.470613,0.03555,0.0000,2.453958e+06,286.808333,49.316389,11.410,220.0,1.000
